In [1]:
import geopandas as gpd
import mercantile
import requests
import os

from tqdm.notebook import tqdm

In [2]:
geojson = gpd.read_file("data/tile_indices_z18/tiles_dissolved_bboxes_buffer1m.geojsonl.json")
geojson.query("id == '36'")

,fid,id,zoom,x_tile,y_tile_ud,y_tile_du,minx_coor,miny_coor,maxx_coor,maxy_coor,width,height,area,perimeter,geometry
9,10,36,18,136278,92393,169750,795862.338506,5.912863e+06,796015.212562,5.913016e+06,1070.118396,1222.992453,1.308747e+06,4586.221697,"POLYGON ((7.14934 46.81416, 7.14934 46.81416, ..."


In [3]:
for zoomlevel in [15, 16, 17, 18, 19, 20, 21]:
    print(f"{zoomlevel = }")

    for row in geojson.query("id == '36'").bounds.iterrows():
        
        bounds = list(row[1].values)
        tiles_generator = mercantile.tiles(*bounds, zooms=zoomlevel)

        tile_ids = [tile_id for tile_id in tiles_generator]
        
        for tile_id in tqdm(tile_ids):
            x = tile_id.x
            y = tile_id.y
            z = tile_id.z

            if os.path.exists(f"data/test_titiler_download/{z}/{x}/{y}.png"):
                continue

            if not os.path.isdir(f"data/test_titiler_download/{z}"):
                os.makedirs(f"data/test_titiler_download/{z}")
            if not os.path.isdir(f"data/test_titiler_download/{z}/{x}"):
                os.makedirs(f"data/test_titiler_download/{z}/{x}")

            response = requests.get(f"https://titiler.stdl.ch/mosaicjson/tiles/{z}/{x}/{y}.png?url=/data/proj-soils/mosaic_soils_all_v20230601_FS.json&bidx=1&bidx=2&bidx=3&nodata=0&pixel_selection=highest")
            with open(f"data/test_titiler_download/{z}/{x}/{y}.png", "wb") as f:
                f.write(response.content)


zoomlevel = 15


  0%|          | 0/4 [00:00<?, ?it/s]

zoomlevel = 16


  0%|          | 0/9 [00:00<?, ?it/s]

zoomlevel = 17


...

  0%|          | 0/288 [00:00<?, ?it/s]

zoomlevel = 20


  0%|          | 0/1020 [00:00<?, ?it/s]

zoomlevel = 21


  0%|          | 0/3828 [00:00<?, ?it/s]